<a href="https://colab.research.google.com/github/daquarti/AI/blob/main/Sentimental_analysis_CHATBOT_usuarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.menos de 48hs de evolución
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tqdm

In [ ]:
path = "/content/drive/My Drive/NLP/Sentence_similarity/"

In [ ]:
import pickle 
with open(path + 'reclamos.pickle', 'rb') as handle:
    reclamos = pickle.load(handle)

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


In [ ]:
!pip install tensorflow-text

     |████████████████████████████████| 3.4MB 12.6MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
 

In [ ]:
# The 16-language multilingual module is the default but feel free
# to pick others from the list and compare the results.
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual/3' #@param ['https://tfhub.dev/google/universal-sentence-encoder-multilingual/3', 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3']

model = hub.load(module_url)

def embed_text(input):
  return model(input)

In [ ]:
from tqdm import tqdm

In [ ]:
lista = []
for x in tqdm(list_complain):
  emb = embed_text (x)[0].numpy()
  lista.append (emb)

100%|██████████| 43271/43271 [11:46<00:00, 61.22it/s]


In [ ]:
kmeans = KMeans(n_clusters=30, random_state=0)
cluster = kmeans.fit_predict(embedding)
etiquetas = kmeans.labels_
len (etiquetas)

1390

In [ ]:
data = {'text': reclamos, 'cluster': cluster}
df = pd.DataFrame.from_dict(data)

In [ ]:
df.cluster.value_counts()

4    442
8    351
0    268
3    130
1    81 
5    53 
2    37 
7    15 
9    8  
6    5  
Name: cluster, dtype: int64

In [ ]:
df[df.cluster == 4]

,text,cluster
2,Hola buenas tardes! El viernes se activo el protocolo Por sospecha de covid. Desde ese momento estoy esperando que vengan a realizar mi hisopado por favor!!!! Graciela Ben Desde ya gracias,4
9,"Yo te envío la foto, pero no podes responderme semejante cosa. Como me voy a registrar sin apellido? Me realizaron el alta, receta de medicamentos y me los vendieron porque tienen mi apellido. La profesional estaría en la luna de Valencia! Enviado desde Yahoo Mail con Android El lun., sept. 28, 2020 a las 10:44 a. m., Info UMA<info@uma-health.com> escribió: Hola José Luis! Las órdenes se generan con los datos que proporcionaste a la hora de hacer el registro en la aplicación, si no figura con apellido es porque al momento de registrarte en ÜMA no señalaste este dato. Para poder corregirlo vamos a requerir que nos hagas llegar una fotografía de tu DNI. Saludos | | | ÜMA Team Email: info@uma-health.com Web: https://uma-health.com/ _________________________________________ | | | | El lun., 28 de sep. de 2020 a la(s) 09:18, jose luis esposito (jl.espo@yahoo.com.ar) escribió: Buen dia.Por la presente me dirijo a uds.a los efectos de solicitarle me vuelvan a confeccionar orden para tomografía pulmonar (neumonia) realizada incorrectamente sin apellido. En el alta de covid que me confeccionaron consta el mismo. Extrañamente e irresponsablemente y sin criterio la doctora que me atendió hoy me pasa esta dirección de mail en lugar de anular la anterior y hacerla correctamente. Son muchas las malas atenciones que me han brindado, como hacerme ir por mis propios medios a atenderme al Anchorena, con sospecha de neumonia. Afiliado 0250066100. Union personal. Espero por favor prota resolucion. Gracias. José Luis Esposito Enviado desde Yahoo Mail con Android",4
14,"Buenas tardes, me llamo Maria Agustina Collado y me dirijo a ustedes para exigir la entrega del resultado de mi hisopado, el cual me hice el día Viernes 18 de Septiembre 2020. Según la técnica que me lo hizo, el resultado estaría para el domingo 20 pero hoy estamos a JUEVES 24 DE SEPTIEMBRE y no hay noticias. Ya inicie como tres reclamos distintos en la obra social Accord Salud pero sigo sin tener respuesta. Entiendo el momento que estamos atravesando y que todo puede retrasarse pero una semana me parece una falta de respeto hacia mi, que en caso de no tener COVID me impide tomar el antibiótico correspondiente, y mi familia, quienes no pueden salir a trabajar. Espero una respuesta, Collado Maria Agustina 36170531",4
19,Loco son una mierda como atenciÃ³n,4
22,"La aplicación tiene problemas, no me da los permisos de micrófono y cámara, aunque yo sí los habilité. No puedo concretar una video llamada con el médico, yo los veo y escucho y ellos no. Estoy a la espera del isopado en mi domicilio, cumpliendo el aislamiento. Solicito una solución a la brevedad. Atentamente Bárbara Brest DNI 28210828",4
24,"hola mi nombre es matias fernandez dni 34791561, hice una video llamada el lunes por sospecha de covid. hoy estoy llamando desde las 10.00 am y no logro comunicarme tendran otra solucion?",4
25,Buen dia : la sra Diaz Magali posee DNI :35126189.-,4
27,"Buenas, envio este correo porque hice todos los pasos a seguir que indica la obra social cuando hay síntomas compatiblrs con posible covid. Descargué la aplicación Gerdanna Salud, por la cual pedí la consulta médica online y el médico que me atendió me dijo q llame al 49599530 para que me autoricen el hisopado. En esta llamada, me dijeron que si o si debía comunicarme con ""Uma"". Seguí los pasos que Uma va diciendo pero nunca llegan más que a decir que ya me registré, que espere un código en mi WhatsApp (código que no llegó nunca durante unas cuantas horas), hasta llamé a la misma (al número que sale en WhatsApp) pero también dice q está apagado o fuera de servicio.. No puedo comunicarme ya que ninguno responde, y lo que busco es, como me dijeron de mi obra social UTA, es la autorización para realizar el hisopado.",4
28,Buenas tardes q

In [ ]:
df.to_csv (path + 'cluster_k30.csv', index = False)

In [ ]:
cluster_labels = {'21': 'Trello', '0': 'UMA', '23':'spam', '6': 'celu', '22': 'recetas', '14': 'celu', '8': 'spam', '16': 'solicitud de atención médica', '15': 'saludo', '25': 'usabilidad de la app_link_acceso', '28': 'datos_filiatorios', '27': 'celu', '19': 'mix', '18': 'síntomas_solicitud_de_atención', '17': 'solicitar_turno', '9': 'recetas', '20': 'mix', '5': 'mix', '24': 'consultas_hisopados', '12': 'mix', '1': 'mix', '7': 'mix', '10': 'spam', '3': 'hisopados_covid', '13': 'sucio', '29': 'correccion_datos_filiatorios', '2': '2', '26': 'busquedas_laborales', '4': 'mix', '11': 'mix' }

In [ ]:
import pandas as pd
df_clean = pd.read_csv(path + 'cluster_k30.csv',
                 lineterminator='\n')

In [ ]:
spam = [21, 23, 6, 14, 8, 15, 27, 0]
recetas = [22, 9]
atencion_medica = [18, 16, 17]
app = [25, 28, 29]
covid = [12, 3, 24]
busquedas_laborales = [26]

In [ ]:
df = df_clean
df.cluster = df.cluster.replace(recetas, 'recetas')
df.cluster = df.cluster.replace(spam, 'spam')
df.cluster = df.cluster.replace(covid, 'covid')
df.cluster = df.cluster.replace(app, 'app')
df.cluster = df.cluster.replace(26, 'busquedas_laborales')
df.cluster = df.cluster.replace(atencion_medica, 'atencion_medica')

In [ ]:
df_soporte = df[df.cluster.isin(['covid', 'spam', 'app', 'atencion_medica', 'busquedas_laborales', 'recetas'])]

In [ ]:
df_soporte.cluster.value_counts(normalize=True)

covid                  0.242547
spam                   0.216802
app                    0.191057
atencion_medica        0.153117
busquedas_laborales    0.111111
recetas                0.085366
Name: cluster, dtype: float64

In [ ]:
df_soporte.to_csv (path + 'df_soporte', index = False)

In [ ]:
df_pendiente = df[df.cluster.isin(['covid', 'spam', 'app', 'atencion_medica', 'busquedas_laborales', 'recetas'])]

In [ ]:
import numpy as np
mask = np.logical_not(df.cluster.isin(['covid', 'spam', 'app', 'atencion_medica', 'busquedas_laborales', 'recetas']))


In [ ]:
df_pendientes = df[mask]

In [ ]:
pendientes = df_pendientes.text.to_list()

In [ ]:
import pickle
with open(path + 'pendientes.pickle', 'wb') as handle:
    pickle.dump(pendientes, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
df[df.cluster == 26]

,text,cluster
59,Hola! Muy buenas noches. Necesito saber el valor de las guardias! Y la cantidad mínima de horas para poder ser Prestadora de Servicio de Salud de Psicólogia. Desde ya muchas gracias! Saludos Cordiales! Muy atentamente. Lic. Nancy Guzmán,26
81,"Buenas tardes , mi nombre es Maria Victoria Segade romay soy mÃ©dica de familia y quisiera averiguar sobre la propuesta laboral de UMA, muchas gracias. Enviado desde mi iPhone",26
119,Estimados Preciso información para dar atención pediatría por sistema UMA gracias Dr. Miguel Maldonado Mn 134981,26
140,"A quien corresponda Mi nombre es Carla Daniela Medina Villafuerte soy medico cirujano y me gustaria poder trabajar para su empresa, cuales son los documentos q tengo que enviar para completar la solicitud Gracias de antemano",26
190,"Buenos días, soy médica neuróloga. Estoy interesada en sumarme a su equipo de consultas por telemedicina. Estoy a su disposición, Saludos cordiales Natali Bein",26
210,"Buenas tardes, pedí un turno en UMa a través de Accord salud. En la página de ustedes seleccione el turno, me indico que se reservó correctamente pero no sé dónde tengo que ingresar cuando sea el turno y no encuentro información al respecto. Muchas gracias -- Marina Zanetti",26
212,"Buenas noches, me presento, mi nombre es Sara Yvone Bravo, soy Médica con Matrícula Nacional (Argentina). Me gustaría recibir información acerca de la incorporación de médicos para asistencia sanitaria por medio de la modalidad de teleconsulta. Sin otro particular, me despido muy atentamente. Sara Yvone Bravo Médica M.N N° 175521",26
230,"buenas tardes , soy medico y me gustaria informacion de como ingresar como proveedor de servicio para realizar telemedicina . muchas gracias",26
270,Buenas era para solicitar informaciÃ³n sobre el trabajo de telemedicina. Gracias lindo dia Enviado desde mi iPhone,26
300,Buenas noches. Me gustaría recibir información sobre la oferta de empleo para médico general. Gracias.,26


In [ ]:
df.to_csv (path + 'cluster.csv', index = False)

In [ ]:
import pandas as pd
df_clean = pd.read_csv(path + 'cluster.csv',
                 lineterminator='\n')

In [ ]:
df_clean.cluster.nunique()

30

In [ ]:
d

NameError: ignored

In [ ]:
lista_mix = []
for k, v in cluster_labels.items():
  if v == 'mix':
    lista_mix.append (k) 

In [ ]:
lista_mix = list(map(int, lista_mix))

In [ ]:
lista_mix

[19, 20, 5, 12, 1, 7, 4, 11]

In [ ]:
df_mix = df[df.cluster.isin (lista_mix)]

In [ ]:
mix_text = df_mix.text.to_list()

In [ ]:
mix_text

['Hola, soy un contacto estrecho. como hago para hacerme el test ? ',
 'buenas tardes quería saber si a travez de esta aplicación se podía pedir la receta para una medicación que tomo regularmente y si me pueden mandar un Link porque tengo el contacto pero no se como entrar a la pagina ..muchas gracias ',
 'Quiero tratar de ingresar a la aplicaciÃ³n y no me deja como puedo reiniciar la aplicacion ',
 'No puedo acceder al historial y descargar una receta. Mi celular es 1535899800. Accord salud  Espero que lo puedan solucionar.  Muchas gracias ',
 'Hola bueno días yo tengo la plataforma de UMA pero como cambie de número de celular tengo que volver hacer de nuevo para poder usar UMA me pide un código pero no me llega ningún al WhatsApp ',
 'Quisiera consultar con una ginecóloga para ver que medicación me pueden recetar. ',
 'Loco son una mierda como atenciÃ³n ',
 'Me pide contraseÃ±a ',
 'La aplicación tiene problemas, no me da los permisos de micrófono y cámara, aunque yo sí los habilité

In [ ]:
df.loc[df['cluster'].isin(spam)]

,text,cluster


In [ ]:
df.cluster.value_counts()

cambio    151
11        94 
4         89 
26        82 
2         80 
29        75 
13        72 
3         69 
10        61 
1         58 
7         58 
12        56 
24        54 
5         51 
20        50 
9         45 
17        45 
18        40 
19        39 
28        34 
25        32 
16        28 
22        18 
0         9  
Name: cluster, dtype: int64

In [ ]:
spam

3        Enviado desde mi smartphone Samsung Galaxy.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

UMAP

In [ ]:
!pip install umap-learn

In [ ]:
import umap
reducer = umap.UMAP()

In [ ]:
embedding = umap.UMAP(n_components=4).fit_transform(lista)

In [ ]:
embedding[1]

array([ 2.6873183,  8.637245 , -0.31437  ,  0.4750196], dtype=float32)

In [ ]:
!pip install hdbscan

     |████████████████████████████████| 6.4MB 11.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.27-cp37-cp37m-linux_x86_64.whl size=2311707 sha256=ebbfde8882ed956afaa78a25eb372226480529bc023c811e9e36b4bcc193594a
  Stored in directory: /root/.cache/pip/wheels/42/63/fb/314ad6c3b270887a3ecb588b8e5aac50b0fad38ff89bb6dff2
Successfully built hdbscan


In [ ]:
import hdbscan

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=4)
cluster_labels = clusterer.fit_predict(embedding)
data = {'text': reclamos, 'cluster': cluster_labels}
df = pd.DataFrame.from_dict(data)

In [ ]:
dic = {'63': 'empleo', '110': 'sintomas', '149': 'covid', '28': 'spam' }

In [ ]:
df[df.cluster == 56]

,text,cluster
14,"Buenas tardes, me llamo Maria Agustina Collado y me dirijo a ustedes para exigir la entrega del resultado de mi hisopado, el cual me hice el día Viernes 18 de Septiembre 2020. Según la técnica que me lo hizo, el resultado estaría para el domingo 20 pero hoy estamos a JUEVES 24 DE SEPTIEMBRE y no hay noticias. Ya inicie como tres reclamos distintos en la obra social Accord Salud pero sigo sin tener respuesta. Entiendo el momento que estamos atravesando y que todo puede retrasarse pero una semana me parece una falta de respeto hacia mi, que en caso de no tener COVID me impide tomar el antibiótico correspondiente, y mi familia, quienes no pueden salir a trabajar. Espero una respuesta, Collado Maria Agustina 36170531",56
146,"Hola, ayer me comunique con mi obra social (Unión personal) para ver cómo tenía que hacer con UMA ya que un médico me había dicho que me iba a hisopar, no vinieron a hacérmelo, y la segunda vez que me comunico me dicen que me dan por positivo epidemiológico. De la obra social me dijeron que no sabían de nadie que los daban por positivo sin un estudio que lo respalde. Me recomendaron volver a hacer la consulta y pedir el hisopado, a lo cual la médica me dijo que ahora se hace así. Sin embargo, ayer me llega un correo de voz pidiéndome que me comunique para coordinar el estudio y que me contacte con ellos. No tengo manera de comunicarme ya que no cuento con ningún número telefónico. Y por este medio nunca me contestaron. Mi número de teléfono es 11-3594-5454. Atte, Camila Carloni",56
152,"Buenas tardes.. con respecto a mi mail anterior y un llamado de ustedes, diría que son muy pocos eficaces y una total falta de respeto, criterio y profesionalidad . Porque a la fecha nunca vinieron a hacerme el hisopado y obviamente nunca sabré si me contagie de COVID 19 . Esto habla muy mal del servicio que prestan . Es lamentable tener que enviar un mensaje con estas palabras.. y los voy a reportar a Accord Salud de la cual soy afiliada .. porque estamos a casi 15 días y jamás hubo una respuesta a mis decenas de llamados diciendo porque no venían .. lamentable contar con un servicio deficiente como el que dicen prestar . El El jue, 13 ago. 2020 a la(s) 16:50, <ofeliaramy@gmail.com> escribió: > Buenas tardes Mi nombre es Ofelia.. Soy una pte. de riesgo x estar > desde el sábado tengo dolor de garganta .. cefalea y estos días se agregó > un estado gripal.. y desde el lunes Tengo la orden de mi nefrólogo para el > hisopado y hice la videoconsulta con un médico de UMA . llegamos a jueves > con un supuesto turno para que se acerquen a mi domicilio y hasta ahora > nada .. me canse de llamar a Emergencias y solo encuentro evasivas.. > apelo al sentido común de ustedes para que intervengan en la situacion.. > adjunto consulta realizada . Muchas gracias",56
200,"Buenas noches mi nombre es Adriana Marisa Zambrano soy Personal Policial (esencial), estoy con diagnóstico presuntivo (sospechoso co) desde el día 18/08/2020 , ya tengo la orden de hisopado y aún no han venido. Tengo la aplicación cuidar inhabilitada por los síntomas que aún estoy cursando: tos, dolor y ardor de garganta, dolor de espalda y perdida de gusto o sabor. Mi obra social es ioma, necesito que me realicen la toma de muestra de hisopado para poder obtener el resultado y volver a mi trabajo. Necesito que me digan a dónde debo realizar el reclamo.Desde ya muchas gracias.",56
211,Hola buenas tardes soy Lopez Celeste y quiero cancelar para que me hagan el hisopado ya que quiero esperar los 3 dias que me dijo la doctora Raquel Eva Patigore ya que me comunique con mi mama y decimos esperar gracias,56
214,"Hola me acaba de llegar un mensaje de voz para coordinar el hisopado. Es de un número privado. Me pueden pasar un número de teléfono así yo me comunico? El lun., 10 de agosto de 2020 16:33, Camila Carloni < camilacarloni6@gmail.com> escribió: > Hola. Yo el viernes 7/8 hice una consulta por contacto con una persona con > covid positivo. En esa consulta, co

In [ ]:
df.cluster.value_counts()

-1     345
 34    100
 25    97 
 0     81 
 35    73 
 23    58 
 52    39 
 1     37 
 56    33 
 36    25 
 17    23 
 48    21 
 54    20 
 24    20 
 8     19 
 16    18 
 39    17 
 11    16 
 9     16 
 4     15 
 40    14 
 13    14 
 26    14 
 12    14 
 46    13 
 42    12 
 33    12 
 5     12 
 30    12 
 38    11 
 50    11 
 7     10 
 22    10 
 28    10 
 29    10 
 44    10 
 18    9  
 3     8  
 27    7  
 21    7  
 20    7  
 32    7  
 51    6  
 49    6  
 10    6  
 43    6  
 41    5  
 37    5  
 14    5  
 45    5  
 47    5  
 6     5  
 15    5  
 31    5  
 19    5  
 53    5  
 2     5  
 55    4  
Name: cluster, dtype: int64

In [ ]:
dic = {'34': 'hisopado', '25': 'recetas', '35': 'empleo', '52': 'atencion_medica', '36': 'app', '48': 'atención medica', '56': 'hisopado', '17': 'spam', '33': 'hisopado' }

In [ ]:
recetas = 25
spam = 17
covid = [34, 56, 33]
empleo = 35
app = 36
atencion_medica = [52, 48]

In [ ]:
df.cluster = df.cluster.replace(recetas, 'recetas')

In [ ]:
df.cluster = df.cluster.replace(spam, 'spam')

In [ ]:
df.cluster = df.cluster.replace(covid, 'covid')

In [ ]:
df.cluster = df.cluster.replace(app, 'app')

In [ ]:
df.cluster = df.cluster.replace(empleo, 'busquedas_laborales')

In [ ]:
df.cluster = df.cluster.replace(atencion_medica, 'atencion_medica')

In [ ]:
df[df.cluster == 33]

,text,cluster


In [ ]:
df_scan = df[df.cluster.isin(['covid', 'spam', 'app', 'atencion_medica', 'busquedas_laborales', 'recetas'])]

In [ ]:
df_scan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 423 entries, 1 to 1386
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     423 non-null    object
 1   cluster  423 non-null    object
dtypes: object(2)
memory usage: 9.9+ KB


In [ ]:
df_final = df_soporte.append(df_scan)

In [ ]:
df_final_drop = df_final.drop_duplicates(subset=['text'])

In [ ]:
df_final_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 792 entries, 1 to 1342
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     792 non-null    object
 1   cluster  792 non-null    object
dtypes: object(2)
memory usage: 18.6+ KB


In [ ]:
df_soporte.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 738 entries, 1 to 1389
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     738 non-null    object
 1   cluster  738 non-null    object
dtypes: object(2)
memory usage: 17.3+ KB


In [ ]:
df_final_drop.to_csv (path + 'df_soporte_final_drop', index = False)

In [ ]:
df_final_drop.sample(10)

,text,cluster
421,hola solicito informacion.. soy medico y estoy interesada para dar consultas on line. Gracias,atencion_medica
30,consulta medico para el dia se hoy,atencion_medica
896,"sromeropasman@gmail.com solicita acceso a un elemento:\r \r Firma\r https://docs.google.com/document/d/1MXh_eAPvZiLBzn924CQtzgKcF0gk5B-tukxld_cPfag/edit?usp=sharing&userstoinvite=sromeropasman@gmail.com&ts=5ee8e585\r \r Eres el propietario de este documento. Para que este usuario pueda acceder, \r haz clic en el vínculo que se encuentra arriba y agrégalo como colaborador \r o visualizador.\r \r",spam
1279,solicitar médico,atencion_medica
791,"[image: Google]\r Alguien accedió a tu cuenta en un nuevo dispositivo\r \r \r info@uma-health.com\r Alguien acaba de acceder a tu cuenta de Google desde un nuevo dispositivo\r Redmi Note 8. Te enviamos este correo electrónico para asegurarnos de que\r hayas sido tú.\r Ver actividad\r <https://accounts.google.com/AccountChooser?Email=info@uma-health.com&continue=https://myaccount.google.com/alert/nt/1583855437000?rfn%3D31%26rfnc%3D1%26eid%3D-3318618317628206568%26et%3D0%26anexp%3Dhsc-control_b---f1>\r Te enviamos este correo electrónico para notificarte acerca de cambios\r importantes en tu cuenta y en los servicios de Google.\r © 2020 Google LLC, 1600 Amphitheatre Parkway, Mountain View, CA 94043, USA\r",spam
592,"Soy docente de la Provincia, mi nombre es Vilma Laura Vilalta, DNI: 23924490, vivo en la calle A1111, en Dock Sud en el barrio de casas verdes, queda entre 12 de Octubre y Lamadrid.Mi telÃ©fono es 1561380734",app
426,Buenas tardes Soy Alejandro Ernesto Abal dni 29200498 Necesito el certificado del hisopado por Covid positivo para presentar en el trabajo. Aguardo certificado. Saludos,covid
978,"Buenos días, soy médica de familia , estaría interesada en la propuesta laboral de telemedicina . Atte , Dra Segade romay María Victoria Enviado desde mi iPhone",busquedas_laborales
1073,Hace cuatro días que ando con este problema vino un médico de emergencia y me dió una medicación Pantometyl cada 12hs dos días estaba bien pero volví a tener los mismos síntomas aún tomando el remedio,atencion_medica
231,Buenas mi nombre es Mercado Norma dni: 29200370 solicito saber el resultado de mi hisopado covid por favor.,covid


In [ ]:
pendientes = df_pendientes.text.to_list ()

In [ ]:
path = "/content/drive/My Drive/NLP/GPT2/autonomous/"

In [ ]:
df = pd.read_csv (path + 'comentarios_dataset.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 901885 entries, 0 to 901884
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     901885 non-null  int64 
 1   complain_text  50965 non-null   object
 2   doc_eval       283660 non-null  object
 3   uma_eval       283660 non-null  object
dtypes: int64(1), object(3)
memory usage: 27.5+ MB


In [ ]:
df = df.dropna(subset=['complain_text'], how = 'all')

In [ ]:
list_complain = df.complain_text.drop_duplicates().to_list()

In [ ]:
len (list_complain)

43271

In [ ]:


import pickle

#with open(path + 'embedding_reclamos.pickle', 'wb') as handle:
#    pickle.dump(lista, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle 
with open(path + 'embedding_reclamos.pickle', 'rb') as handle:
    embedding_use = pickle.load(handle)

In [ ]:
embedding_use[0].shape

(512,)

In [ ]:
embedding = umap.UMAP(n_components=4).fit_transform(embedding_use)

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=4)
cluster_labels = clusterer.fit_predict(embedding)
data = {'text': list_complain, 'cluster': cluster_labels}
df = pd.DataFrame.from_dict(data)

In [ ]:
df.to_csv ( path + 'reclamos_hdbscan.csv', index = False)

In [ ]:
grouped = df.groupby('cluster')
df2 = grouped.apply(lambda x: x.sample(4))


In [ ]:
recetas = [900, 710]
spam = []
covid = [1185, 789, 1210, 925]
empleo = []
app = [476, 426, 670]
atencion_medica = [306, 1244, 1058, 1257, 1315, 1109, 977, 764, 1043, 735, 566, 946, 1091]

In [ ]:
df = pd.read_csv (path + 'reclamos_hdbscan.csv')

In [ ]:
df[df.cluster == -1].sample(20)

,text,cluster
24802,"Estoy esperando la ambulancia mi padre no da más de dolores, si no la envían dentro de los 20' hago la demanda x abandono de persona",-1
19481,"No me atendió el médico , estuve esperando que me atienda y nada !!!!",-1
3368,Me llama el médico y cuando entro no me comunico,-1
789,Quisiera saber si van a asistir me lo más pronto posible Gracias,-1
31776,Quiero que me atienda el.medicoo x favor,-1
26529,Del día 14 q estoy esperando q asistan para el isopado,-1
16869,No puedo ingresar al link. Dice que el doctor dio de baja la reunión.,-1
4961,"La atención es para mí hija pero no aparece sus datos, también es ioma, me podrían informar",-1
9476,Por favor! Estoy en espera hace 10 minutos y me dice que ya me van a atender !,-1
38321,Esperando que alguien me responda para consulta a domicilio e hisopado según lo acordado ayer con el Dr. Nicolás Sanz,-1


In [ ]:
df.cluster.value_counts().head(30)

-1       20606
 467     388  
 1244    306  
 0       254  
 900     240  
 426     231  
 1058    183  
 1257    165  
 1234    159  
 1315    142  
 1185    139  
 1109    127  
 977     127  
 710     122  
 789     113  
 764     111  
 1043    106  
 735     105  
 350     99   
 670     98   
 925     94   
 1091    91   
 215     89   
 946     88   
 1210    87   
 772     83   
 163     79   
 867     79   
 1202    78   
 566     77   
Name: cluster, dtype: int64

In [ ]:
df[df.cluster == 134].sample(20)

,text,cluster
34876,Se cayó la app y No me atendieron,134
36584,"No se puede conectar, atiendo y no ninguna videollamada, y lo que escribo no se envia, aplicación de mierda!! Llamenmen!! Para eso tienen mi numero, y existe el WhatsApp, porque UMA no funciona!",134
25304,No me esta funcionando la aplicacion,134
32083,Consulta atienden por la app?,134
37787,Está trabada la aplicación,134
10841,La app no sirveNadie me atendió,134
1932,Hola anda mal la aplicación\n,134
18846,La app no fugura cuando me van a atender.,134
4317,Que demora tiene debo esperar con la app abierta?,134
25124,"No sé. Es dudoso,.me aprecen varios nombres con el link... voy a esperar mejor, no entiendombien la APP",134


In [ ]:
df.cluster = df.cluster.replace(recetas, 'recetas')

In [ ]:
df.cluster = df.cluster.replace(spam, 'spam')

In [ ]:
df.cluster = df.cluster.replace(covid, 'covid')

In [ ]:
df.cluster = df.cluster.replace(app, 'app')

In [ ]:
df.cluster = df.cluster.replace(empleo, 'busquedas_laborales')

In [ ]:
df.cluster = df.cluster.replace(atencion_medica, 'atencion_medica')

In [ ]:
df_scan_reclamos = df[df.cluster.isin(['covid', 'spam', 'app', 'atencion_medica', 'busquedas_laborales', 'recetas'])]

In [ ]:
df_scan_reclamos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2787 entries, 20 to 43270
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     2787 non-null   object
 1   cluster  2787 non-null   object
dtypes: object(2)
memory usage: 65.3+ KB


In [ ]:
df_scan_reclamos.sample(20)

,text,cluster
34582,Estoy esperando el medico desde ayer,atencion_medica
42083,"Pasaron más de 10 minutos desde que apareció el cartel ""En breve será atendido"" y ningún médico me atendió.",atencion_medica
33,"No me atiende nadie,la nena tiene 39;5 de fiebre",atencion_medica
41306,Queria saber si me podria realizar nuevamente el anterior certificado ya que al intentar descargarlo no se lee nada. Y si podria darme un certificado de alta medica ya que no me encuentro mas difonica,recetas
15023,Me dice que no es posible conectarse,app
40572,Estoy esperando que me atienda el médico hace más de 10 minutos,atencion_medica
32377,Estoy esperando un médico,atencion_medica
23521,Recién hoy vinieron a hacerme el hizopado. Cuando esté el resultado te llamo. Gracias,covid
21214,No funciona bien la app,app
24303,Estuve esperando y nadie te atiende,atencion_medica


In [ ]:
df = pd.read_csv (path + 'df_soporte_final_drop')

In [ ]:
df.head()

,text,cluster
0,"Buenos días , no pude ingresar a la sala, no cargaba recetas, no podía cerrar recetas marcaba error CMR 009..eSTUVE UNA HORA SIN PODER ATENDER.} Saludos cordiales Elena Zevallos",recetas
1,Hola buenas tardes! El viernes se activo el protocolo Por sospecha de covid. Desde ese momento estoy esperando que vengan a realizar mi hisopado por favor!!!! Graciela Ben Desde ya gracias,covid
2,Enviado desde mi smartphone Samsung Galaxy.,spam
3,Maria veronica rodriguez,app
4,Qiero hacer una video llamada me tira error el DNIEnviado desde mi smartphone Samsung Galaxy.,spam


In [ ]:
df_final2 = df.append(df_scan_reclamos)

In [ ]:
df_final2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3579 entries, 0 to 43270
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     3579 non-null   object
 1   cluster  3579 non-null   object
dtypes: object(2)
memory usage: 83.9+ KB


In [ ]:
df_final2.cluster.value_counts()

atencion_medica        1753
covid                  652 
app                    506 
recetas                457 
spam                   124 
busquedas_laborales    87  
Name: cluster, dtype: int64

In [ ]:
df_final2.sample(10)

,text,cluster
8855,Yo soy positivo de covid19.me mandaron está aplicación pero nadie me da una respuesta\n,covid
19384,Hace 15 minutos estoy esperando y no me atiende el doctor,atencion_medica
2835,No me atienden estoy esperando hace mucho tiempo,atencion_medica
544,Monica Beratz Dni 17589977 Domicilio. M. Acevedo 1551 pb f. Tel 1133209413,app
40277,Buenos días .el 27 de agosto me vinieron aser el hisopado y hasta la fecha no me dieron el resultado,covid
6436,Estoy esperando un médico me dijeron 8 minutos y ya pasó ese tiempo,atencion_medica
37773,Sigo esperando. Nadie se comunicó.,atencion_medica
15405,Sigo esperando un médico por favor,atencion_medica
14197,Quiero hablar con un médico,atencion_medica
7998,Estoy a la espera de que me llamen y no lo hacen,atencion_medica


In [ ]:
#df_final2.to_csv (path + 'df_soporte_mails_complain.csv', index = False)

In [ ]:
df = pd.read_csv (path + 'df_soporte_mails_complain.csv')

In [ ]:
df.cluster.value_counts (normalize = True)

atencion_medica        0.489802
covid                  0.182174
app                    0.141380
recetas                0.127689
spam                   0.034647
busquedas_laborales    0.024308
Name: cluster, dtype: float64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3579 entries, 0 to 3578
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     3579 non-null   object
 1   cluster  3579 non-null   object
dtypes: object(2)
memory usage: 56.0+ KB


In [ ]:
df.cluster.value_counts()

atencion_medica        1753
covid                  652 
app                    506 
recetas                457 
spam                   124 
busquedas_laborales    87  
Name: cluster, dtype: int64

In [ ]:
df2 = pd.read_csv (path + 'complain.csv')

In [ ]:
df2 = df2.dropna()

In [ ]:
lista_topic = df2.topic.to_list()

In [ ]:
lista_javi = df2['complain'].to_list()

In [ ]:
lista = []
for x in tqdm(lista_javi):
  emb = embed_text (x)[0].numpy()
  lista.append (emb)


100%|██████████| 8813/8813 [01:44<00:00, 84.55it/s]


In [ ]:
embedding = umap.UMAP(n_components=4).fit_transform(lista)

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=4)
cluster_labels = clusterer.fit_predict(embedding)
data = {'text': lista_javi, 'cluster': cluster_labels, 'topic': lista_topic}
df = pd.DataFrame.from_dict(data)

In [ ]:
recetas = [13, 125 ]
spam = []
covid = [ ]
empleo = []
app = [39]
atencion_medica = [63, 52, 126, 4, 164, 234]

In [ ]:
df.to_csv (path + 'javi_reclamos.csv', index = False)

In [ ]:
df[df.cluster == 39]

,text,cluster,topic
696,no pude ni comenzar a hablar con medico pq no escuchaba...,245,CONNECTION_PROBLEM
1155,me intente comunicar con dos medicos el cual ninguno me llamo,245,NO_CALL
1273,intente comunicarme con el medico y me da error,245,CONNECTION_PROBLEM
1544,no pude hablar con ningun medico.,245,NO_CALL
1580,nunca me puedo comunicar con un medico,245,CONNECTION_PROBLEM
1758,no me comunican con un medico\n,245,NO_CALL
1937,"nunca pude comunicarme con el doctor, nunca fui atendida",245,NO_CALL
2051,no me llega el wats para hablar con el doc.\n\n,245,NO_CALL
2058,no puedo recibir mi consulta medica!!!\n,245,NO_CALL
2115,no puedo hablar con un medico,245,OTHER


In [ ]:
df.cluster.value_counts().head(20)

-1      3184
 13     446 
 63     123 
 52     116 
 126    84  
 4      73  
 164    71  
 234    67  
 8      66  
 125    66  
 228    63  
 293    55  
 18     51  
 32     48  
 39     42  
 245    41  
 149    39  
 99     39  
 28     38  
 68     38  
Name: cluster, dtype: int64